### Spark Session

Entry point to PySpark's functionality within a program. 

In [1]:
### Building SparkSession

from pyspark.sql import SparkSession   ### SparkSession entry point located in pyspark.sql pkg, providing functionality for data transformation

spark = (SparkSession
         .builder     ### Builder pattern abstraction for constructing a sparksession, where we chain the methods to configure the entry point.
         .appName("Analyzing the vocabulary of Pride and Prejudice.") ### Relevant appName helping in identifying which programs run on the Spark cluster
         .getOrCreate()) ### Program works in both interactive or batch mode by avoiding creating a new session if one already exists.

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/08 00:43:55 WARN Utils: Your hostname, OnePiece, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
26/01/08 00:43:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/08 00:43:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark.sparkContext ### For using low-level RDD

""" SparkSession is the wrapper around sparkContext which uses dataframe as it is more versatile and fast as the main datastructure over lower-level RDD"""



' SparkSession is the wrapper around sparkContext which uses dataframe as it is more versatile and fast as the main datastructure over lower-level RDD'

### Setting the Log Level

Monitoring PySpark jobs is an important part of developing. PySpark has many levels of logging, from nothing to full description of everything happening in cluster. pyspark shell defaults on WARN, which is a bit chatty.  And, PySpark program defaults to INFO level which is oversharing. 

In [ ]:
### Changing the log level to keyword

spark.sparkContext.setLogLevel("KEYWORD")

### Keywords: WARN, ALL, INFO, DEBUG, TRACE, OFF, etc.

In [3]:
dir(spark.read)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_df',
 '_jreader',
 '_set_opts',
 '_spark',
 'csv',
 'format',
 'jdbc',
 'json',
 'load',
 'option',
 'options',
 'orc',
 'parquet',
 'schema',
 'table',
 'text',
 'xml']